We want the neural net to be given the same size volume at each episode.

First, we check to see if the branch is at least as big as the target volume before we sample nodes from it.

In [7]:
from swc_io import get_fnames_and_abspath_from_dir, swc_to_nparray

TARGET_DIM_SIZE = 25  # size in pixels for all x,y,z dims
min_swc_dim_span = TARGET_DIM_SIZE+2  # some padding to make things easy

BRANCHES_DIR = "../data/03_human_branches_splitted"

fnames, abs_paths = get_fnames_and_abspath_from_dir(BRANCHES_DIR)
for file in abs_paths[:1]:  # TODO: get rid of slice
    print("f", file)
    nodes = swc_to_nparray(file)
    print(nodes)


f /home/shit/bin/allen/vaa3d_tools/hackathon/profundo/data/03_human_branches_splitted/038_06092018.ano.swc_root2270433_child2270432.swc
[[2270433.      21322.889   11924.667    5228.753]
 [2270432.      21322.297   11924.667    5228.605]
 [2270431.      21322.1     11923.63     5228.556]
 [2270430.      21322.      11920.223    5228.309]
 [2270429.      21321.334   11914.667    5226.531]
 [2270428.      21316.666   11909.556    5226.309]
 [2270427.      21314.666   11905.333    5224.531]
 [2270426.      21312.666   11901.333    5224.531]
 [2270425.      21311.334   11894.667    5224.531]
 [2270424.      21310.      11891.333    5223.197]
 [2270423.      21310.223   11884.223    5222.53 ]
 [2270422.      21312.      11878.223    5222.308]
 [2270421.      21312.      11873.777    5220.752]
 [2270420.      21312.      11858.       5220.53 ]
 [2270419.      21310.223   11852.       5220.308]
 [2270418.      21309.777   11846.223    5218.53 ]
 [2270417.      21308.223   11841.777    5218.53

In [3]:
print(len(fnames))

61228
